- Register env with ray.tune.registery
- training config
- can access policy with Algorithm.get_policy()
- check evaluation options in config
- check checkpoint options in config
- for fast env small model PPO: increase num_env_runners, maybe eneable vectorization for inference
- check batch size

In [3]:
from rleplus.examples.registry import register_all
register_all()

In [5]:
import ray
from ray import air, tune
from ray.rllib.algorithms.ppo import PPOConfig

In [35]:
env_setup = {
    "env": "BBrightEnv",
    "csv": False,
    "verbose": False,
    "output": "./tmp/",
    "num_workers": 1,
    "num_gpus": 0,
    "alg": "PPO",
    "use-lstm": False
}


config = (
    PPOConfig()
    .environment(
        env=env_setup["env"],
        env_config=env_setup,
    )
    .framework("torch")
    .training(
        gamma=0.95,
        lr=0.003,
        kl_coeff=0.3,
        train_batch_size=4000,
        sgd_minibatch_size=128,
        vf_loss_coeff=0.01,
        use_critic=True,
        use_gae=True,
        model={
            "use_lstm": True,
            "vf_share_layers": False,
        },
        _enable_learner_api=True,
    )
    .rl_module(_enable_rl_module_api=True)
    .framework(
        # to use tensorflow, you'll need install it first,
        # then set framework="tf2" and eager_tracing=True (for fast exec)
        framework="torch",
    )
    .resources(num_gpus=env_setup["num_gpus"])
    .rollouts(
        num_rollout_workers=env_setup["num_workers"],
        rollout_fragment_length="auto",
    )
    
)

In [36]:
ray.shutdown()

In [39]:
ray.init()

register_all()

algo = config.build()

for i in range(10):
    result = algo.train()
    print(type(result), result)

    if i % 5 == 0:
        checkpoint = algo.save()
        print("checkpoint saved at", checkpoint)

ray.shutdown()

/home/l4/.cache/pypoetry/virtualenvs/rl-energyplus-hi-YszVVEjS-py3.10/lib/python3.10/site-packages/ray/_private/node.py:1308: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-08-07_10-23-06_034116_440765/logs/gcs_server.out' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/l4/.cache/pypoetry/virtualenvs/rl-energyplus-hi-YszVVEjS-py3.10/lib/python3.10/site-packages/ray/_private/node.py:1308: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-08-07_10-23-06_034116_440765/logs/gcs_server.err' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/l4/.cache/pypoetry/virtualenvs/rl-energyplus-hi-YszVVEjS-py3.10/lib/python3.10/site-packages/ray/_private/node.py:1313: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-08-07_10-23-06_034116_440765/logs/monitor.out' mode='a' encoding='utf-8'>
  self.start_monitor()
/home/l4/.cache/pypoetry/virtualenvs/rl-energyplus-hi-YszVVEjS-py3.10/lib/python3.10/

<class 'dict'> {'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 2560.0, 'num_env_steps_trained': 4000.0, 'total_loss': 0.14328483889805788}, 'default_policy': {'total_loss': 0.14328483889805788, 'policy_loss': 0.023461725840226134, 'vf_loss': 8.35616523661512, 'vf_loss_unclipped': 311.47320954343104, 'vf_explained_var': 9.502502197914935e-05, 'entropy': 1.401684580965245, 'mean_kl_loss': 0.12087156340675535, 'default_optimizer_lr': 0.0030000000000000005, 'curr_lr': 0.003, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.45000001788139343}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_s

/usr/lib/python3.10/subprocess.py:1072: ResourceWarning: subprocess 674926 is still running
  _warn("subprocess %s is still running" % self.pid,
